
<table align="center" >
    <thead>
        <tr>
            <th>
                <a  href="http://www.fsdmfes.ac.ma/">
                    <img src="https://github.com/m-elkhou/Object-Detection/blob/master/assets/usmba.png?raw=1" width=70px />
                </a>
            </th>
            <th>
                <a  href="http://www.fsdmfes.ac.ma/">
                    <img src="https://github.com/m-elkhou/Object-Detection/blob/master/assets/fsdm.png?raw=1" width=75px />
                </a>
            </th>
            <th>
                <a href="https://www.univ-paris13.fr/">
                    <img src="https://github.com/m-elkhou/Object-Detection/blob/master/assets/uspn.png?raw=1" width=150px/>
                </a>
            </th>
            <th>
                <a href="http://www.imperium-media.com/">
                    <img src="https://github.com/m-elkhou/Object-Detection/blob/master/assets/imperium_media.png?raw=1" width=70px/>
                </a>
            </th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align:center;">
                <a   href="http://www.fsdmfes.ac.ma/">USMBA</a>
            </td>
            <td style="text-align:center;">
              <a   href="http://www.fsdmfes.ac.ma/">FSDM</a>
            </td>
            <td style="text-align:center;">
                <a  href="https://www.univ-paris13.fr/">USPN</a>
            </td>
            <td style="text-align:center;">
                <a  href="http://www.imperium-media.com/">IMPERIUM MEDIA</a>
            </td>
        </tr>
    </tbody>
</table>


***

#  **Object Detection with Deep Learning** #

***

<div align="left" >
    <a href="https://colab.research.google.com/github/m-elkhou/Object-Detection/blob/master/Implementation.ipynb">
        <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" />
    </a>
    <a href="https://github.com/m-elkhou/Object-Detection/blob/master/Implementation.ipynb">
        <img align="left" src="https://badgen.net/badge/icon/Open%20in%20GitHub?icon=github&label" alt="Open in GitHub" title="View source on GitHub" />
    </a>
    <br/>
</div>

## Implementation Details:
Here, we are going to explain the implementation steps for  different object detection models. To train the model on the custom dataset, standard implementation steps would be as below:
1. Data Acquisition
2. Data Labelling
3. Data Preparation
4. Model Training
5. Evaluation / Results analysis

### Data Acquisition

Data acquisition is a required step when you want to train the model on the custom dataset. It will require more than 100 images with the object for which you want to train the model. Each image should have good quality, contains object/objects at least once (to be detected). No image should be repetitive within the dataset, the more the variety of images, the more extensive the training becomes.

Single Shot Detectron (SSD) and Faster R-CNN require images with objects only, but in the case of YOLO, negative samples are also being used for the training and validation.

Here, we are going to showcase the demo for the model training, and prediction of the custom object which is the logo of the brand VIVO. We have collected ~200 images for the training.

### Data Labelling

For the custom dataset, we need to label objects from each of the images. For object detection, the model requires certain details of the objects that are to be detected from the image and those are the X-axis, y-axis, height, and width of the object within the image.

Run below command to install labelImg

```python
pip install labelImg
```
You can also refer to https://github.com/tzutalin/labelImg for the reference. The labelImg tool provides a user interface that allows a user to draw bounding boxes and captures the x-axis and y-axis along with the height and width of the bounding box. This also provides a feature to label multiple objects within a single image. labelImg stores axis values in a separate XML file at the same location where the image is present.

**Data Preparation & Model Training**

### Faster R-CNN:
#### Data Preparation:
1. Create a text file from the xml file which is generated from the labelImg where txt file where each image path and it’s relative bounding boxes co-ordinate in new line which is in the following format and move it to the cloned repository :
```
filepath,xmin,ymin,xmax,ymax,class_name
```  

#### Training

1. In order to implement the object detection using Faster R-CNN first we need to clone this GitHub repository:
```
git clone https://github.com/kbardool/keras-frcnn
```
2. Move the train_images and test_images folder to the cloned repository
3. Now open the terminal in the repository folder and install requirements.txt file.
4. We can train the model using Faster R-CNN through executing this line in the terminal:
```
python train_frcnn.py -o simple -p (path to txt file)
```

#### Inference:
1. Now put the testing images in the folder named test_images and in the same folder of Faster R-CNN and execute the following line in the terminal:
python test_frcnn.py -p test_images

### SSD:

We have referred the following link for the implementation of object detection using SSD:
https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
    
#### Data Preparation:

1. Created .pbtxt file containing label names and it’s id.
2. Then we converted all labelImg xml file to csv file using xml_to_csv.py.
3. We then converted the CSV file to .record file format using generate_tfrecord.py.

#### Training:

1. Then we had downloaded the pretrained model of ssd_inception and also created a config file according to tutorial with the following changes in the config file:
```config
num_classes: 1
type: 'ssd_inception_v2' # Set to the name of your chosen pre- \ trained
model
fine_tune_checkpoint: "pre-trained-model/model.ckpt" # Path to \ extracted
files of pre-trained model
train_input_reader: { \
tf_record_input_reader { \
input_path: "annotations/train.record" # Path to training \
TFRecord file
}
label_map_path: "annotations/label_map.pbtxt" # Path to label \ map
file
}
```

2. We can train the model through executing this line in the terminal:
```
python train.py --logtostderr --train_dir=training -- \ pipeline_config_path=training/ssd_inception_v2_coco.config
```

#### Inference:
1. Copy the TensorFlow/models/research/object_detection/export_inference_graph.py script and paste it straight into your training_demo folder.
2. Then using the highest checkpoint file of the model and generate .pb file by executing this line in the terminal:
```
python export_inference_graph.py --input_type image_tensor -- \ pipeline_config_path training/ssd_inception_v2_coco.confi -- \ trained_checkpoint_prefix training/model.ckpt-13302 - \ output_directory trained-inference- \ graphs/output_inference_graph_v1.pb
```
3. Using this object_detection code we can predict the test images for SSD:


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
sys.path.append("..")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Path to frozen detection graph .pb file
PATH_TO_CKPT = '.../frozen_inference_graph.pb'

# Path to label map file
PATH_TO_LABELS = '.../label_map.pbtxt' 

# Path to image
PATH_TO_IMAGE = '.../test_images/394.jpg'

# Number of classes the object detector can identify
NUM_CLASSES = 1

# Load the label map.
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = \ label_map_util.convert_label_map_to_categories(label_map, \ max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    sess = tf.Session(graph=detection_graph)
    
# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Each box represents a part of the image where a particular \ object was detected
detection_boxes = \ detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
detection_scores = \ detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = \ detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = \ detection_graph.get_tensor_by_name('num_detections:0')

image = cv2.imread(PATH_TO_IMAGE)
image_expanded = np.expand_dims(image, axis=0)

# Perform the actual detection by running the model with the image \ as input
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})
    
vis_util.visualize_boxes_and_labels_on_image_array( \
    image, \
    np.squeeze(boxes), \
    np.squeeze(classes).astype(np.int32), \
    np.squeeze(scores), \
    category_index, \
    use_normalized_coordinates=True, \
    line_thickness=8, \
    min_score_thresh=0.60)

# All the results have been drawn on image. Now display the image.
cv2.imshow('Object detector', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Result Analysis

In our use case, we have focused on the detection of the logo of the brand VIVO. We have annotated 150 images for the training. Each image has an average of 3 objects. We have captured training images from the different sports events in order to keep the variety of background within the train images. Also, we have captured 50 of the images, with quite a similar background. However, Faster R-CNN and SSD do not consider negative samples (samples without objects) for the training.

For the comparative analysis as a first observation, we have trained the object detection model for 2K steps/ iteration on 150 images.

Sample training images are as shown below:

In [4]:
import tarfile
tf = tarfile.open("samples.tar.gz")
# tf.extractall()

FileNotFoundError: [Errno 2] No such file or directory: 'samples.tar.gz'